In [1]:
import pandas as pd
import networkx as nx
import community as community_louvain
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from numpy import dot
from numpy.linalg import norm


In [2]:
def cosine(a, b):
    return dot(a, b)/(norm(a)*norm(b))

In [3]:
df = pd.read_csv('CrawlData/data/reprocessed_data/relationship.csv')
df.head()

,id1,id2
0,8674,2382
1,8674,12512
2,8674,10317
3,8674,1098
4,8674,14219


In [4]:
data = pd.read_csv('CrawlData/data/cleaned.csv')
data['string']=data[['Workplaces', 'College','School', 'Places','Gender' ]].agg(' '.join, axis=1)
data.drop(['Workplaces', 'College','School', 'Places','Gender' ], axis=1, inplace=True)
data

,id,string
0,2,works at esquel group no_college no_school bie...
1,3,long xuyen quan tri mang truong trung cap cong...
2,4,no_workplaces no_college no_school thanh hoa t...
3,5,works at binh duong viet nammay 16 2020 presen...
4,6,ben tre can tho can tho university truong thpt...
...,...,...
13069,19718,ho chi minh city vietnam tc kt nn thpt nguyen ...
13070,19719,no_workplaces no_college no_school hanoi vietn...
13071,19720,no_workplaces quan tri kinh doanh bvu truong d...
13072,19721,no_workplaces no_college thpt nguyen chi thanh...


In [5]:
all_nodes = set(list(df['id1']) + list(df['id2']))

In [6]:
len(all_nodes)

20913

In [7]:
G = nx.Graph()

In [8]:
G.add_nodes_from(all_nodes)
for i, r in df.iterrows():
    G.add_edge(r['id1'], r['id2'], weight = 1)

In [9]:
cleaned_id = list(data['id'])
nodes = list(G.nodes())
for n in nodes:
    if n not in cleaned_id:
        G.remove_node(n)

In [10]:
# labels = nx.get_edge_attributes(G, 'weight')
# ll = []
# for key, value in labels.items():
#     ll.append({'id1':key[0], 'id2':key[1], 'weight': value})
# pd.DataFrame(ll).to_csv('CrawlData/data/reprocessed_data/relationship_content_based.csv', index=False)

In [11]:
partition = community_louvain.best_partition(G)

In [12]:
values = list(partition.values())
print('Number of communities: ', len(np.unique(values)))

Number of communities:  3038


In [13]:
data['value'] = values

In [14]:
community_index = 15
data_community = data[data.value == community_index].reset_index(drop=True)
data_community

,id,string,value
0,21,works at rmit student ambassadors team s a toc...,15
1,34,no_workplaces an toan thong tin clc truong dai...,15
2,41,no_workplaces cong nghe thong tin truong dai h...,15
3,82,no_workplaces no_college no_school ho chi minh...,15
4,90,no_workplaces truong dai hoc cong nghe thong t...,15
...,...,...,...
1135,19645,no_workplaces software engineering truong dai ...,15
1136,19655,lcd khoa ke toan hoc vien tai chinh chuyen nga...,15
1137,19681,no_workplaces truong thpt phan chau trinh tien...,15
1138,19705,junior at lampart co ltd junior at lampart co ...,15


In [15]:
indices = pd.Series(data_community.index,data_community.id)

In [16]:
vectorizer = CountVectorizer()
X1 = vectorizer.fit_transform(data_community["string"])
vectors = X1.toarray()

In [17]:
target_id = 99
target_vector = vectors[indices[target_id]]

In [18]:
cosine_map = []
for i, v in zip(list(data_community['id']), vectors):
    cosine_map.append((i, cosine(target_vector, v)))

In [19]:
friend_list =set(nx.neighbors(G, target_id))

In [23]:
ranked_list = sorted(cosine_map, key=lambda x: x[1], reverse=True)[1:len(friend_list) + 10]
id_ranked_list = [i[0] for i in ranked_list]

In [24]:
len(set(friend_list).intersection(set(id_ranked_list)))/len(id_ranked_list)

0.1320754716981132

In [25]:
len(set(friend_list).intersection(set(id_ranked_list)))

14